<a href="https://colab.research.google.com/github/preetamjumech/LLM/blob/main/Chat_with_Video_File_using_Qwen2_VL_Model_03_11_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install accelerate flash_attn
!pip install qwen_vl_utils av

  Using cached flash_attn-2.6.3.tar.gz (2.6 MB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for flash_attn
  Running setup.py clean for flash_attn
Failed to build flash_attn
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (flash_attn)


In [3]:
!pip uninstall transformers
!pip install -U git+https://github.com/huggingface/transformers

Found existing installation: transformers 4.47.0.dev0
Uninstalling transformers-4.47.0.dev0:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.10/dist-packages/transformers-4.47.0.dev0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/transformers/*
Proceed (Y/n)? Y
  Successfully uninstalled transformers-4.47.0.dev0
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-l8ejm2h1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-l8ejm2h1
  Resolved https://github.com/huggingface/transformers to commit 33868a057c02f0368ba63bd1edb746be38fe3d90
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.47.0.dev0-py3-none-any.whl size=10057149 sha256=9ee98049a922ef20e8982d8201240c37d47c64b311ee6b0305486a27b73a8f78
  Stored in directory:

In [4]:
import transformers
print(transformers.__version__)

4.47.0.dev0


In [1]:
from transformers import  AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch

In [5]:
from transformers import Qwen2VLForConditionalGeneration

In [6]:
model_name = "Qwen/Qwen2-VL-2B-Instruct"

In [7]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_name, torch_dtype=torch.bfloat16,  device_map="auto"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
processor = AutoProcessor.from_pretrained(model_name)

In [9]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "video",
                "video": "/content/sample.mp4",
                "max_pixels": 360 * 420,
                "fps": 1.0,
            },
            {"type": "text", "text": "Explain the video here?"},
        ],
    }
]

In [10]:
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

In [ ]:
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)

qwen-vl-utils using torchvision to read video.


In [ ]:
inputs = inputs.to("cuda")

In [ ]:
generated_ids = model.generate(**inputs, max_new_tokens=512)

In [ ]:
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]

In [ ]:
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)


In [ ]:
print(output_text)